# Homework 2: training pipeline

This code will test your homework 2 solutions by using them in a complete ML pipeline. You should run this code in order to tune your model and save your model weights (which will also be uploaded as part of your solution)

In [173]:
# Download the training data from the homework2 folder:
# unzip using tar xzvvf nsynth_subset.tar.gz
# (this is a small subset of the "nsynth" dataset: https://magenta.tensorflow.org/datasets/nsynth)

In [1]:
import homework2

### Install and Load Required Libraries  

In [2]:
# !pip install librosa
# !pip install torch
# !pip install glob
# !pip install numpy

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as nnF
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import librosa
import random
import glob

In [3]:
BATCH_SIZE = 16
torch.use_deterministic_algorithms(True)

In [4]:
if not len(homework2.audio_paths):
    print("You probably need to set the dataroot folder correctly")

In [5]:
# Some helper functions. These are the same as what the autograder runs.

In [5]:
# Split dataset into train / valid / test
def split_data(waveforms, labels, train_ratio=0.7, valid_ratio=0.15):
    assert(train_ratio + valid_ratio < 1)
    test_ratio = 1 - (train_ratio + valid_ratio)
    N = len(waveforms)
    Ntrain = int(N * train_ratio)
    Nvalid = int(N * valid_ratio)
    Ntest = int(N * test_ratio)
    Wtrain = waveforms[:Ntrain]
    Wvalid = waveforms[Ntrain:Ntrain + Nvalid]
    Wtest = waveforms[Ntrain + Nvalid:]
    ytrain = labels[:Ntrain]
    yvalid = labels[Ntrain:Ntrain + Nvalid]
    ytest = labels[Ntrain + Nvalid:]
    return Wtrain,Wvalid,Wtest,ytrain,yvalid,ytest

In [6]:
def process_data(W, feature_function):
    return [feature_function(path) for path in W]

In [7]:
class InstrumentDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        features = self.features[idx]
        label = self.labels[idx]

        return features, torch.tensor(label, dtype=torch.long)

In [8]:
class Loaders():
    def __init__(self, waveforms, labels, feature_function, seed = 0):
        torch.manual_seed(seed)
        random.seed(seed)
        self.Wtrain, self.Wvalid, self.Wtest, self.ytrain, self.yvalid, self.ytest = split_data(waveforms, labels)
        
        self.Xtrain = process_data(self.Wtrain, feature_function)
        self.Xvalid = process_data(self.Wvalid, feature_function)
        self.Xtest = process_data(self.Wtest, feature_function)
        
        self.dataTrain = InstrumentDataset(self.Xtrain, self.ytrain)
        self.dataValid = InstrumentDataset(self.Xvalid, self.yvalid)
        self.dataTest = InstrumentDataset(self.Xtest, self.ytest)
        
        self.loaderTrain = DataLoader(self.dataTrain, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
        self.loaderValid = DataLoader(self.dataValid, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
        self.loaderTest = DataLoader(self.dataTest, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [9]:
class Pipeline():
    def __init__(self, module, learning_rate, seed = 0):
        # These two lines will (mostly) make things deterministic.
        # You're welcome to modify them to try to get a better solution.
        torch.manual_seed(seed)
        random.seed(seed)

        self.device = torch.device("cpu") # Can change this if you have a GPU, but the autograder will use CPU
        self.criterion = nn.CrossEntropyLoss()
        
        self.model = module.to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

    def evaluate(self, loader, which = "valid"):
        self.model.eval()

        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                outputs = self.model(inputs)
                #loss = criterion(outputs, labels) # validation loss

                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        acc = correct / total
        
        return acc
    
    def train(self, loaders,
          num_epochs=1, # Train for a single epoch by default
          model_path=None): # (Optionally) provide a path to save the best model
        val_acc = 0
        best_val_acc = 0
        for epoch in range(num_epochs):
            self.model.train()
            
            losses = []

            for inputs, labels in loaders.loaderTrain:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                losses.append(float(loss))
            
            self.model.eval()
            val_acc = self.evaluate(loaders.loaderValid)
            print("Epoch " + str(epoch) + ", loss = " + str(sum(losses)/len(losses)) +\
                  ", validation accuracy = " + str(val_acc))

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                if (model_path):
                    torch.save(self.model.state_dict(), model_path)
        print("Final validation accuracy = " + str(val_acc) + ", best = " + str(best_val_acc))
        return val_acc, best_val_acc

    def load(self, path):
        self.model.load_state_dict(torch.load(path, weights_only=True))

In [185]:
# The function below is the basis of how the autograder tests your code. Try to understand this one.

In [10]:
def test(waveforms, labels, feature_func, classifier, learning_rate, path):
    print("Extracting features...")
    test_loaders = Loaders(waveforms, labels, feature_func)
    test_pipeline = Pipeline(classifier, learning_rate)
    
    # Note: the autograder will not run this line: it will just load your saved model (next line)
    acc, best_acc = test_pipeline.train(test_loaders, 10, path)
    
    test_pipeline.load(path)
    test_acc = test_pipeline.evaluate(test_loaders.loaderTest)
    print("Test accuracy = " + str(test_acc))

In [187]:
# 1. Paths, labels, waveforms

In [11]:
# Once you've written the corresponding code in homework2.py, print these out or visualize them if you want
homework2.waveforms
homework2.labels

[0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,


In [189]:
# 2. MFCC

In [15]:
test(homework2.waveforms,
     homework2.labels,
     homework2.extract_mfcc,
     homework2.MLPClassifier(),
     0.0001,
     "best_mlp_model.weights")

Extracting features...


KeyboardInterrupt: 

In [191]:
# 3. Spectrogram

In [13]:
test(homework2.waveforms,
     homework2.labels,
     homework2.extract_spec,
     homework2.SimpleCNN(),
     0.0001,
     "best_spec_model.weights")

Extracting features...
Epoch 0, loss = 0.6166174991263284, validation accuracy = 0.8780487804878049
Epoch 1, loss = 0.5419693829284774, validation accuracy = 0.8861788617886179
Epoch 2, loss = 0.5081521752807829, validation accuracy = 0.9024390243902439
Epoch 3, loss = 0.4824598754445712, validation accuracy = 0.9186991869918699
Epoch 4, loss = 0.45449155734644997, validation accuracy = 0.926829268292683
Epoch 5, loss = 0.42432256125741535, validation accuracy = 0.959349593495935
Epoch 6, loss = 0.39340511212746304, validation accuracy = 0.959349593495935
Epoch 7, loss = 0.3633945360779762, validation accuracy = 0.975609756097561
Epoch 8, loss = 0.3358999399675263, validation accuracy = 0.983739837398374
Epoch 9, loss = 0.31280837539169526, validation accuracy = 0.983739837398374
Final validation accuracy = 0.983739837398374, best = 0.983739837398374
Test accuracy = 0.9274193548387096


In [ ]:
# 4. Mel-spectrogram

In [14]:
test(homework2.waveforms,
     homework2.labels,
     homework2.extract_mel,
     homework2.SimpleCNN(),
     0.0001,
     "best_mel_model.weights")

Extracting features...
Epoch 0, loss = 0.4973374828696251, validation accuracy = 0.8211382113821138
Epoch 1, loss = 0.34611181749237907, validation accuracy = 0.8861788617886179
Epoch 2, loss = 0.2898188887370957, validation accuracy = 0.926829268292683
Epoch 3, loss = 0.2535473306973775, validation accuracy = 0.959349593495935
Epoch 4, loss = 0.22609401080343458, validation accuracy = 0.983739837398374
Epoch 5, loss = 0.20433735785384974, validation accuracy = 0.991869918699187
Epoch 6, loss = 0.186005265555448, validation accuracy = 1.0
Epoch 7, loss = 0.16972103238933617, validation accuracy = 1.0
Epoch 8, loss = 0.15508649725880888, validation accuracy = 1.0
Epoch 9, loss = 0.141966519670354, validation accuracy = 1.0
Final validation accuracy = 1.0, best = 1.0
Test accuracy = 0.9596774193548387


In [ ]:
# 5. Constant-Q

In [15]:
test(homework2.waveforms,
     homework2.labels,
     homework2.extract_q,
     homework2.SimpleCNN(),
     0.0001,
     "best_q_model.weights")

Extracting features...
Epoch 0, loss = 0.5847891097267469, validation accuracy = 0.7317073170731707
Epoch 1, loss = 0.49428534176614547, validation accuracy = 0.8373983739837398
Epoch 2, loss = 0.43594976100656724, validation accuracy = 0.8699186991869918
Epoch 3, loss = 0.39206450349754757, validation accuracy = 0.8780487804878049
Epoch 4, loss = 0.35594940144154763, validation accuracy = 0.9024390243902439
Epoch 5, loss = 0.32545389814509285, validation accuracy = 0.9349593495934959
Epoch 6, loss = 0.2994835161500507, validation accuracy = 0.9349593495934959
Epoch 7, loss = 0.27698606956336236, validation accuracy = 0.943089430894309
Epoch 8, loss = 0.2568066798978382, validation accuracy = 0.9512195121951219
Epoch 9, loss = 0.23882457738121352, validation accuracy = 0.959349593495935
Final validation accuracy = 0.959349593495935, best = 0.959349593495935
Test accuracy = 0.9516129032258065


In [ ]:
# 6. Pitch shift

In [16]:
test(homework2.augmented_waveforms,
     homework2.augmented_labels,
     homework2.extract_q,
     homework2.SimpleCNN(),
     0.0001,
     "best_augmented_model.weights")

Extracting features...
Epoch 0, loss = 0.5398875634979319, validation accuracy = 0.8807588075880759
Epoch 1, loss = 0.4076669115435194, validation accuracy = 0.9349593495934959
Epoch 2, loss = 0.35079168059207777, validation accuracy = 0.948509485094851
Epoch 3, loss = 0.3158196496466796, validation accuracy = 0.967479674796748
Epoch 4, loss = 0.2890690853474317, validation accuracy = 0.978319783197832
Epoch 5, loss = 0.2659092732463722, validation accuracy = 0.978319783197832
Epoch 6, loss = 0.24521819853948223, validation accuracy = 0.981029810298103
Epoch 7, loss = 0.22640807636910015, validation accuracy = 0.981029810298103
Epoch 8, loss = 0.20916573682592976, validation accuracy = 0.986449864498645
Epoch 9, loss = 0.19282537622860185, validation accuracy = 0.991869918699187
Final validation accuracy = 0.991869918699187, best = 0.991869918699187
Test accuracy = 0.9972972972972973


In [ ]:
# 7. Extend your model to handle four classes and creatively improve its performance

In [30]:
test(homework2.waveforms,
     homework2.labels_7,
     homework2.feature_func_7,
     homework2.model_7,
     0.0001,
     "best_model_7.weights")

Extracting features...
Epoch 0, loss = 0.16960719372663233, validation accuracy = 0.8861788617886179
Epoch 1, loss = 0.1866714141021172, validation accuracy = 0.8861788617886179
Epoch 2, loss = 0.21030323973132503, validation accuracy = 0.8780487804878049
Epoch 3, loss = 0.20177413647373518, validation accuracy = 0.8780487804878049
Epoch 4, loss = 0.20604266060723198, validation accuracy = 0.8861788617886179
Epoch 5, loss = 0.20390293478137916, validation accuracy = 0.8780487804878049
Epoch 6, loss = 0.21148579609062937, validation accuracy = 0.8780487804878049
Epoch 7, loss = 0.20243709182573688, validation accuracy = 0.8943089430894309
Epoch 8, loss = 0.22781602976222834, validation accuracy = 0.8943089430894309
Epoch 9, loss = 0.21770358230504724, validation accuracy = 0.8943089430894309
Final validation accuracy = 0.8943089430894309, best = 0.8943089430894309
Test accuracy = 0.9516129032258065
